## DIA 089: Agregar Autenticación de Usuarios (Login y Registro) en Flask

Hoy llevaremos tu aplicación a otro nivel añadiendo autenticación de usuarios, es decir:

Un sistema de registro

Un formulario de inicio de sesión

Control de sesiones con Flask

Esto permitirá que cada usuario pueda hacer sus propias predicciones y más adelante podrías guardar un historial por usuario. 🔐

✅ ¿Qué aprenderás hoy?

📌 Crear y validar usuarios con SQLite y Flask-SQLAlchemy.

📌 Usar contraseñas cifradas con werkzeug.security.

📌 Controlar sesiones con Flask-Login.

🧩 1. 📦 Requisitos
Instala estas librerías si no las tienes:

bash
Copiar
Editar
pip install flask flask_sqlalchemy flask_login
🗂️ Estructura del Proyecto
pgsql
Copiar
Editar
titanic_app/
├── app.py
├── model.pkl
├── db.sqlite3
├── templates/
│   ├── index.html
│   ├── login.html
│   ├── register.html
│   ├── historial.html
│   └── grafica.html
🧠 2. Código Completo de app.py con Autenticación
python
Copiar
Editar
from flask import Flask, render_template, request, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, login_user, login_required, logout_user, UserMixin, current_user
from werkzeug.security import generate_password_hash, check_password_hash
import joblib
import numpy as np
import datetime

app = Flask(__name__)
app.secret_key = 'secreto123'  # Necesario para sesiones
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
db = SQLAlchemy(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

# MODELOS
class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(100), unique=True)
    password = db.Column(db.String(200))

class Registro(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))
    f1 = db.Column(db.Float)
    f2 = db.Column(db.Float)
    f3 = db.Column(db.Float)
    f4 = db.Column(db.Float)
    prediccion = db.Column(db.String(50))
    fecha = db.Column(db.DateTime, default=datetime.datetime.utcnow)

modelo = joblib.load("model.pkl")

# USUARIO ACTUAL
@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

# RUTAS
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/registro', methods=['GET', 'POST'])
def registro():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        hash_pass = generate_password_hash(password)

        if User.query.filter_by(username=username).first():
            flash("⚠️ Usuario ya existe")
            return redirect(url_for('registro'))

        nuevo = User(username=username, password=hash_pass)
        db.session.add(nuevo)
        db.session.commit()
        flash("✅ Usuario registrado con éxito")
        return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        user = User.query.filter_by(username=request.form['username']).first()
        if user and check_password_hash(user.password, request.form['password']):
            login_user(user)
            return redirect(url_for('prediccion'))
        flash("❌ Credenciales incorrectas")
    return render_template('login.html')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('home'))

@app.route('/prediccion', methods=['GET', 'POST'])
@login_required
def prediccion():
    resultado = ""
    if request.method == 'POST':
        try:
            f1 = float(request.form["f1"])
            f2 = float(request.form["f2"])
            f3 = float(request.form["f3"])
            f4 = float(request.form["f4"])

            datos = np.array([[f1, f2, f3, f4]])
            pred = modelo.predict(datos)[0]

            nuevo_registro = Registro(user_id=current_user.id, f1=f1, f2=f2, f3=f3, f4=f4, prediccion=str(pred))
            db.session.add(nuevo_registro)
            db.session.commit()
            resultado = f"✅ Resultado: Clase {pred}"
        except:
            resultado = "❌ Error en los datos"
    return render_template("index.html", resultado=resultado)

if __name__ == '__main__':
    db.create_all()
    app.run(debug=True)
🖼️ 3. Plantillas HTML necesarias
register.html
html
Copiar
Editar
<h2>Registro</h2>
<form method="POST">
    Usuario: <input name="username"><br>
    Contraseña: <input type="password" name="password"><br>
    <button type="submit">Registrarse</button>
</form>
<a href="/login">¿Ya tienes cuenta?</a>
login.html
html
Copiar
Editar
<h2>Iniciar sesión</h2>
<form method="POST">
    Usuario: <input name="username"><br>
    Contraseña: <input type="password" name="password"><br>
    <button type="submit">Ingresar</button>
</form>
<a href="/registro">Crear cuenta nueva</a>